In [1]:
import numpy as np
from os import listdir, mkdir, path
import matplotlib.pyplot as plt
from matplotlib import colors, cm
from scipy.optimize import curve_fit
from scipy.signal import medfilt
from sklearn import tree
import re
%matplotlib

Using matplotlib backend: TkAgg


Following function shepherds files of various naming conventions into the correct order.

In [2]:
def find_n_order_files(folder1,folder2,folder3,folder4,verbose=False):
    files = listdir(folder1)
    files1 = [ii for ii in files if ((not ii.__contains__('bkg')) & (ii.__contains__('csv')))]
    seq1_1 = [ii for ii in files1 if re.match('Rxn1_[0-9]{4}_dz_bgsub.csv', ii)]
    seq1_2 = [ii for ii in files1 if re.match('Rxn1_350_[0-9]{4}_dz_bgsub.csv', ii)]
    seq1_3 = [ii for ii in files1 if re.match('Rxn1_cooling_[0-9]{4}_dz_bgsub.csv', ii)]
    seq1 = []
    seq1.extend(seq1_1)
    seq1.extend(seq1_2)
    seq1.extend(seq1_3)
    files1 = [path.join(folder1,  ii) for ii in seq1]

    files = listdir(folder2)
    files2 = [ii for ii in files if ((not ii.__contains__('Bckgrd')) & (ii.__contains__('csv')))]
    seq2_1 = [ii for ii in files2 if re.match('Rxn2_125_waxs_[0-9]{4}_dz_bgsub.csv', ii)]
    seq2_2 = [ii for ii in files2 if re.match('Rxn2_350_waxs_[0-9]{4}_dz_bgsub.csv', ii)]
    seq2_3 = [ii for ii in files2 if re.match('Rxn2_350_hold_[0-9]{4}_dz_bgsub.csv', ii)]
    seq2_4 = [ii for ii in files2 if re.match('Rxn2_cool_[0-9]{4}_dz_bgsub.csv', ii)]
    seq2 = []
    seq2.extend(seq2_1)
    seq2.extend(seq2_2)
    seq2.extend(seq2_3)
    seq2.extend(seq2_4)
    files2 = [path.join(folder2,  ii) for ii in seq2]

    files = listdir(folder3)
    files3 = [ii for ii in files if ((not ii.__contains__('Bkg')) & (ii.__contains__('csv')))]
    seq3_1 = [ii for ii in files3 if re.match('R3_125_[0-9]{4}_dz_bgsub.csv', ii)]
    seq3_2 = [ii for ii in files3 if re.match('R3_350_[0-9]{4}_dz_bgsub.csv', ii)]
    seq3_3 = [ii for ii in files3 if re.match('R3_350_II_[0-9]{4}_dz_bgsub.csv', ii)]
    seq3_4 = [ii for ii in files3 if re.match('R3_350_cooling_[0-9]{4}_dz_bgsub.csv', ii)]
    seq3 = []
    seq3.extend(seq3_1)
    seq3.extend(seq3_2)
    seq3.extend(seq3_3)
    seq3.extend(seq3_4)
    files3 = [path.join(folder3,  ii) for ii in seq3]

    files = listdir(folder4)
    files4 = [ii for ii in files if ((not ii.__contains__('BG')) & (ii.__contains__('csv')))]
    seq4_1 = [ii for ii in files4 if re.match('Rxn4_II_350_[0-9]{4}_dz_bgsub.csv', ii)]
    seq4_2 = [ii for ii in files4 if re.match('Rxn4_350_hold_[0-9]{4}_dz_bgsub.csv', ii)]
    seq4 = []
    seq4.extend(seq4_1)
    seq4.extend(seq4_2)
    files4 = [path.join(folder4,  ii) for ii in seq4]

    if verbose:
        print "Sequence lengths in folder 1: %i, %i, %i; expected: 100, 1000, 200" % (len(seq1_1), len(seq1_2), len(seq1_3))
        print "Sequence lengths in folder 2: %i, %i, %i, %i; expected: 100, 1000, 668, 100" % (len(seq2_1), len(seq2_2), len(seq2_3), len(seq2_4))
        print "Sequence lengths in folder 3: %i, %i, %i, %i; expected: 120, 500, 100, 100" % (len(seq3_1), len(seq3_2), len(seq3_3), len(seq3_4))
        print "Sequence lengths in folder 4: %i, %i; expected: 100, 566" % (len(seq4_1), len(seq4_2))
    return files1, files2, files3, files4 #, seq1, seq2, seq3, seq4

Following cell is one-time code for copying files from Sam's portable hard drive to my disk.  Don't execute.

In [ ]:
'''
from shutil import copy as filecopy

folder1 = "/Volumes/FreeAgent GoFlex Drive/SLAC/20170316_processed/R1/sample/raw/"
folder2 = "/Volumes/FreeAgent GoFlex Drive/SLAC/20170316_processed/R2/sample/raw/"
folder3 = "/Volumes/FreeAgent GoFlex Drive/SLAC/20170316_processed/R3/sample/raw/"
folder4 = "/Volumes/FreeAgent GoFlex Drive/SLAC/20170316_processed/R4/sample/raw/"

outfolder1 = "/Users/Amanda/Dropbox/Sam_data_20170316/R1"
outfolder2 = "/Users/Amanda/Dropbox/Sam_data_20170316/R2"
outfolder3 = "/Users/Amanda/Dropbox/Sam_data_20170316/R3"
outfolder4 = "/Users/Amanda/Dropbox/Sam_data_20170316/R4"

def copy_to_disk():
    files1, files2, files3, files4 = find_n_order_files(folder1,folder2,folder3,folder4)
    outfiles1 = [path.join(outfolder1, path.split(ii)[1]) for ii in files1]
    outfiles2 = [path.join(outfolder2, path.split(ii)[1]) for ii in files2]
    outfiles3 = [path.join(outfolder3, path.split(ii)[1]) for ii in files3]
    outfiles4 = [path.join(outfolder4, path.split(ii)[1]) for ii in files4]
    for ii in range(len(files1)):
        filecopy(files1[ii], outfiles1[ii])
    for ii in range(len(files2)):
        filecopy(files2[ii], outfiles2[ii])
    for ii in range(len(files3)):
        filecopy(files3[ii], outfiles3[ii])
    for ii in range(len(files4)):
        filecopy(files4[ii], outfiles4[ii])

copy_to_disk()
'''

For identifying & sequencing files under normal circumstances.

In [3]:
folder1 = "/Users/Amanda/Dropbox/Sam_data_20170316/R1/"
folder2 = "/Users/Amanda/Dropbox/Sam_data_20170316/R2/"
folder3 = "/Users/Amanda/Dropbox/Sam_data_20170316/R3/"
folder4 = "/Users/Amanda/Dropbox/Sam_data_20170316/R4/"

files1, files2, files3, files4 = find_n_order_files(folder1,folder2,folder3,folder4)

Make subfolders, if necessary.

In [4]:
def mk_subfolders():
    for ii in [folder1,folder2,folder3,folder4]:
        if not path.isdir(path.join(ii, 'plots')):
            mkdir(path.join(ii, 'plots'))

mk_subfolders()

In [5]:
def readfiles(filenames):
    nfiles = len(filenames)
    ref_spec = np.loadtxt(filenames[0], delimiter=',')
    ref_q, _ = ref_spec.T
    intensity_image = np.zeros((ref_q.size,nfiles),dtype=float)
    for ii in range(nfiles):
        spectrum = np.loadtxt(filenames[ii], delimiter=',')
        spec_q, spec_I = spectrum.T
        if (spec_q != ref_q).any():
            raise ValueError("Issue!  domain mismatch %s" % filenames[ii])
        intensity_image[:,ii] = spec_I
    return intensity_image, ref_spec

In [6]:
intensity_image1, ref_spec1 = readfiles(files1)
intensity_image2, ref_spec2 = readfiles(files2)
intensity_image3, ref_spec3 = readfiles(files3)
intensity_image4, ref_spec4 = readfiles(files4)

Document the state of the time series before removing outliers.

In [7]:
loc1 = path.join(folder1, 'plots', 'R1')
loc2 = path.join(folder2, 'plots', 'R2')
loc3 = path.join(folder3, 'plots', 'R3')
loc4 = path.join(folder4, 'plots', 'R4')

def waterfall_plot(diff_image, ref, loc=None, title_append=None, log=True):
    nq, nfiles = diff_image.shape
    qq,_ = ref.T
    viridis = plt.get_cmap('viridis')
    cNorm = colors.Normalize(vmin=0, vmax=nfiles)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=viridis)
    fig, ax = plt.subplots()
    ax.plot([qq[0],qq[-1]], [0,0], color='k',lw=1.5)
    for ii in range(nfiles):
        colorVal = scalarMap.to_rgba(ii)
        ax.plot(qq, diff_image[:,ii], color=colorVal, lw=0.5)
    title = 'Waterfall plot'
    if title_append is not None:
        title += title_append
    ax.set_title(title)
    if log:
        ax.set_yscale('log')
        lims = ax.axis()
        ax.axis([lims[0], lims[1], 0.01, lims[3]])
    if loc is not None:
        print loc + '_waterfall.pdf'
        fig.savefig(loc + '_waterfall.pdf')
        #plt.close(fig)

In [8]:
waterfall_plot(intensity_image1, ref_spec1, loc1+'_pre_outlier_removal', ', R1 pre-removal')
#waterfall_plot(intensity_image2, ref_spec2, loc2+'R2_pre_outlier_removal', ', R2 pre-removal')
waterfall_plot(intensity_image3, ref_spec3, loc3+'_pre_outlier_removal', ', R3 pre-removal')
#waterfall_plot(intensity_image4, ref_spec4, loc4+'R4_pre_outlier_removal', ', R4 pre-removal')

/Users/Amanda/Dropbox/Sam_data_20170316/R1/plots/R1_pre_outlier_removal_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_pre_outlier_removal_waterfall.pdf


In [14]:
ref_point1 = [4, 100, 500]

def crude_filter(intensity_image, ref_spec, loc, ref_point=None):
    # plot before
    fig, ax = plt.subplots(1,3)
    ax[0].imshow(np.log(intensity_image.T), cmap='viridis')
    # separate into subtracted and not subtracted
    nq, nfiles = intensity_image.shape
    blank = np.ones((nq, nfiles),dtype=float)
    file_index = np.arange(nfiles, dtype=int)
    ref_q,_ = ref_spec.T
    q_ref_index = min(np.where(ref_q > ref_point[0])[0])
    subtracted = (intensity_image[q_ref_index,:] > ref_point[1]) & (intensity_image[q_ref_index,:] < ref_point[2])
    kept_indices = np.where(subtracted)
    ngood = subtracted.sum()
    subtracted = np.array(subtracted[np.newaxis,:]*blank, dtype=bool)
    good_spectra = intensity_image[subtracted].reshape(nq, ngood)
    if ref_point is not None:
        bad_spectra = intensity_image[~subtracted].reshape(nq, nfiles-ngood)
    # require values be positive
    floor = 0.01
    good_spectra[np.where(good_spectra < floor)] = floor
    if ref_point is not None:
        bad_spectra[np.where(bad_spectra < floor)] = floor
    else:
        print "RuntimeWarning: divide by zero coming up, don't worry about it"
        bad_spectra = np.zeros((nq,1))
    # plot after
    ax[1].imshow(np.log(good_spectra.T), cmap='viridis')
    ax[2].imshow(np.log(bad_spectra.T), cmap='viridis')
    ax[0].set_title('Before cleanup')
    ax[0].axis('off')
    ax[1].set_title('Subtracted, cleaned')
    ax[1].axis('off')
    ax[2].set_title('Unsubtracted, cleaned')
    ax[2].axis('off')
    fig.savefig(loc+"_cleaning.pdf")
    waterfall_plot(good_spectra, ref_spec, loc+"_subtracted", ", cleaned spectra")
    if ref_point is not None:
        waterfall_plot(bad_spectra, ref_spec, loc+"_unsubtracted", ", outlier spectra")
    return good_spectra, bad_spectra, kept_indices

In [15]:
good_spectra1, bad_spectra1, kept_indices = crude_filter(intensity_image1, ref_spec1, loc1, ref_point1)
good_spectra3 = intensity_image3
waterfall_plot(good_spectra3, ref_spec3, loc3+"_subtracted", ", cleaned spectra")

/Users/Amanda/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/Users/Amanda/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  


/Users/Amanda/Dropbox/Sam_data_20170316/R1/plots/R1_subtracted_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R1/plots/R1_unsubtracted_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_subtracted_waterfall.pdf


In [11]:
lambdas1 = [3.09, 3.2, 3.66]
lambdas3 = [2.78, 3.07, 3.22, 3.32, 4.49]

lims1 = [[2.95,3.05,3.4,3.5],[3.45,3.55,3.9,4.0]]
lims3 = [[2.5,2.6,2.95,3.05],[2.95,3.05,3.4,3.5],[4.2,4.3,4.7,4.8]]

def linear(x, a, b):
    return a + b*x

def quadratic(x, a, b, c):
    return a + b*x + c*x**2

def cubic(x, a, b, c, d):
    return a + b*x + c*x**2 + d*x**3

def quartic(x, a, b, c, d, e):
    return a + b*x + c*x**2 + d*x**3 + e*x**3

def quintic(x, a, b, c, d, e, f):
    return a + b*x + c*x**2 + d*x**3 + e*x**3 + f*x**5


def basic_bg_local(image, func, p0, ref_spec, lims, loc):
    qq,_ = ref_spec.T
    nq, nfiles = image.shape
    blank = np.ones(image.shape, dtype=bool)
    subtracted_regions = []
    for ii in range(len(lims)):
        q1 = lims[ii][0]
        q2 = lims[ii][1]
        q3 = lims[ii][2]
        q4 = lims[ii][3]
        locale = ((qq > q1) & (qq < q4))
        feature = ((qq > q2) & (qq < q3))
        periphery = (locale & ~feature)
        qlocale = qq[locale]
        qperiphery = qq[periphery]
        nlocale = locale.sum()
        nfeature = feature.sum()
        nperiphery = periphery.sum()
        locale = image[locale[:,np.newaxis]*blank].reshape(nlocale,nfiles)
        feature = image[feature[:,np.newaxis]*blank].reshape(nfeature,nfiles)
        periphery = image[periphery[:,np.newaxis]*blank].reshape(nperiphery,nfiles)
        waterfall_plot(locale, np.array([qlocale,qlocale]).T, loc+"_local_%i"%ii,', unsubtracted local',False)
        subtracted = np.zeros(locale.shape,dtype=float)
        for jj in range(nfiles):
            popt, _ = curve_fit(func,qperiphery,periphery[:,jj],p0=p0)
            subtracted[:,jj] = locale[:,jj] - func(qlocale,*popt)
        waterfall_plot(subtracted+1, np.array([qlocale,qlocale]).T, loc+"_local_subtracted_%i"%ii,', subtracted local',False)
        subtracted_regions.append([qlocale,subtracted])
    return subtracted_regions

In [13]:
subtracted_regions1 = basic_bg_local(good_spectra1, cubic, [1,0,0,0], ref_spec1, lims1, loc1)
subtracted_regions3 = basic_bg_local(good_spectra3, cubic, [1,0,0,0], ref_spec3, lims3, loc3)

/Users/Amanda/Dropbox/Sam_data_20170316/R1/plots/R1_local_0_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R1/plots/R1_local_subtracted_0_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R1/plots/R1_local_1_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R1/plots/R1_local_subtracted_1_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_local_0_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_local_subtracted_0_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_local_1_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_local_subtracted_1_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_local_2_waterfall.pdf
/Users/Amanda/Dropbox/Sam_data_20170316/R3/plots/R3_local_subtracted_2_waterfall.pdf


In [31]:
def gauss(x, a, x0, sigma):
    #return (2*np.pi*sigma**2)**-0.5 * np.exp(-0.5*((x - x0)/sigma)**2)
    return a * np.exp(-0.5*((x - x0)/sigma)**2)

def residuals(x, y, func, pars):
    return (((y - func(x, *pars))**2).mean())**0.5

def manypeaks(x, *params):
    # params pattern:  a0, x0, s0, a1, x1, s1...
    params = np.array(params)
    npeaks = params.size/3
    y = np.zeros(x.shape,dtype=float)
    for ii in range(npeaks):
        ai,xi,si = params[3*ii:3*ii+3]
        y += gauss(x,ai,xi,si)
    return y

def plotarray(nplots, arrange_vertical=True, sharex=False, sharey=False):
    nplots = int(nplots)
    print nplots
    if nplots == 1:
        fig, ax = plt.subplots()
        return fig, np.array([ax])
    if arrange_vertical:
        nrows = int(np.ceil(nplots**0.5))
        ncols = int(np.ceil(nplots/nrows))
    else:
        ncols = int(np.ceil(nplots**0.5))
        nrows = int(np.ceil(nplots/ncols))
    fig, ax = plt.subplots(nrows,ncols)
    flatax = ax.ravel()
    for ii in range(ax.size):
        if ii > (nplots - 1):
            flatax[ii].axis('off')
    return fig, ax

def sample_plot_array(n, q, data_seq, fit_seq=None):
    fig,ax = plotarray(n, True, True, True)
    flatax = ax.ravel()
    N = len(data_seq)
    indices = np.arange(n,dtype=int)*nfiles/n + int(0.5*nfiles)/n
    print n, N, indices
    for ii in range(n):
        flatax[ii].plot(q, data_seq[:,indices[ii]], c='k',lw=1)
        if fit_seq:
            flatax[ii].plot(q, fit_seq[:,indices[ii]], c='b',lw=2)
        #flatax[ii].axis('off')
    fig.suptitle("Evolution of sequence")
'''
    kk = 0
    for ii in range(ax.shape[0]):
        for jj in range(ax.shape[1]):
            colorVal = scalarMap.to_rgba(indices[kk])
            ax[ii,jj].plot(q, data_seq[:,indices[kk]], c=colorVal,lw=1)
            if fit_seq is not None:
                ax[ii,jj].plot(q, fit_seq[:,indices[kk]], c=colorVal,lw=2)
            if (ii != ax.shape[0]-1):
                ax[ii,jj].get_xaxis().set_ticks([])
            if (jj != 0):
                ax[ii,jj].get_yaxis().set_ticks([])
            #flatax[ii].axis('off')
            kk += 1
'''
 

def sample_plot_waterfall(n, q, data_seq, fit_seq=None):
    fig,ax = plt.subplots()
    nq,nfiles = data_seq.shape
    indices = np.arange(n,dtype=int)*nfiles/n + int(0.5*nfiles)/n
    viridis = plt.get_cmap('viridis')
    cNorm = colors.Normalize(vmin=0, vmax=nfiles)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=viridis)

    for ii in range(n):
        colorVal = scalarMap.to_rgba(indices[ii])
        ax.plot(q, data_seq[:,indices[ii]], color=colorVal, lw=1)
        if fit_seq is not None:
            ax.plot(q, fit_seq[:,indices[ii]], c=colorVal,lw=2)
    fig.suptitle("Evolution of sequence")
    
    
def parameter_trace_display(params):
    npars,nfiles = params.shape
    viridis = plt.get_cmap('viridis')
    cNorm = colors.Normalize(vmin=0, vmax=nfiles)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=viridis)
    for ii in range(npars/3):
        fig,axarray = plt.subplots(2,1)
        a, x0, sigma = params[ii*3:ii*3+3,:]
        for jj in range(nfiles):
            if (a[jj]==0)&(x0[jj]==0)&(sigma[jj]==0):
                continue
            colorVal = scalarMap.to_rgba(jj)
            axarray[0].plot(x0[jj],a[jj]*np.sqrt(2*np.pi)*sigma[jj],ls='none',lw=0.5,marker='.',color=colorVal)
            axarray[1].plot(sigma[jj],a[jj],ls='none',lw=0.5,marker='.',color=colorVal)
        axarray[0].set_xlabel('peak position, q')
        axarray[0].set_ylabel('peak integrated intensity')
        axarray[1].set_xlabel('peak width $\sigma$')
        axarray[1].set_ylabel('peak height')

    

In [ ]:
parameter_trace_display(fit_record_1_0)
parameter_trace_display(fit_record_1_1)
parameter_trace_display(fit_record_3_0)
parameter_trace_display(fit_record_3_1a)
parameter_trace_display(fit_record_3_1b)
parameter_trace_display(fit_record_3_1c)
parameter_trace_display(fit_record_3_2)

In [ ]:
sample_plot_waterfall(16, subtracted_regions1[0][0], subtracted_regions1[0][1])
sample_plot_waterfall(16, subtracted_regions1[1][0], subtracted_regions1[1][1])
sample_plot_waterfall(16, subtracted_regions3[0][0], subtracted_regions3[0][1])
sample_plot_waterfall(16, subtracted_regions3[1][0], subtracted_regions3[1][1])
sample_plot_waterfall(16, subtracted_regions3[2][0], subtracted_regions3[2][1])

In [32]:
p0_1_0 = [70., 3.16, 0.04]
bd_1_0 = [[0., 3.12, 0.02],
          [120., 3.31, 0.07]]

p0_1_1 = [30., 3.68, 0.04]
bd_1_1 = [[0., 3.61, 0.02],
          [55., 3.82, 0.07]]

p0_3_0 = [50., 2.78, 0.025]
bd_3_0 = [[0., 2.74, 0.01],
          [60., 2.81, 0.06]]

p0_3_1a = [75., 3.21, 0.035]
bd_3_1a = [[0., 3.12, 0.02],
           [90., 3.36, 0.07]]

p0_3_1b = [75., 3.21, 0.035, 55., 3.31, 0.035]
bd_3_1b = [[0., 3.12, 0.02, 0., 3.26, 0.02],
           [90., 3.26, 0.07, 70., 3.36, 0.07]]

p0_3_1c = [20., 3.12, 0.035, 75., 3.21, 0.035, 55., 3.31, 0.035]
bd_3_1c = [[0., 3.05, 0.02, 0., 3.16, 0.02, 0., 3.26, 0.02],
           [35., 3.16, 0.07, 90., 3.26, 0.07, 70., 3.36, 0.07]]

p0_3_2 = [22., 4.48, 0.025]
bd_3_2 = [[0., 4.44, 0.01],
          [35., 4.53, 0.06]]

In [33]:
def individual_fit(qq,image,func,p0,bd):
    nq,nfiles = image.shape
    fit_image = np.zeros(image.shape)
    fit_record = np.zeros((len(p0),nfiles))
    failcounter = 0
    for ii in range(nfiles):
        try:
            popt,_ = curve_fit(func,qq,image[:,ii],p0=p0,bounds=bd)
            p0=popt
        except RuntimeError:
            popt = 0*np.array(p0)
            failcounter += 1
        fit_image[:,ii] = func(qq,*popt)
        fit_record[:,ii] = popt
    if failcounter != 0:
        print "%i spectra did not converge to a solution." % failcounter
    else:
        print "All spectra converged to a fit solution."
    return fit_image, fit_record

In [34]:
fit_image_1_0,fit_record_1_0 = individual_fit(subtracted_regions1[0][0],subtracted_regions1[0][1],gauss,p0_1_0,bd_1_0)
fit_image_1_1,fit_record_1_1 = individual_fit(subtracted_regions1[1][0],subtracted_regions1[1][1],gauss,p0_1_1,bd_1_1)
fit_image_3_0,fit_record_3_0 = individual_fit(subtracted_regions3[0][0],subtracted_regions3[0][1],gauss,p0_3_0,bd_3_0)
fit_image_3_1a,fit_record_3_1a = individual_fit(subtracted_regions3[1][0],subtracted_regions3[1][1],gauss,p0_3_1a,bd_3_1a)
fit_image_3_1b,fit_record_3_1b = individual_fit(subtracted_regions3[1][0],subtracted_regions3[1][1],manypeaks,p0_3_1b,bd_3_1b)
fit_image_3_1c,fit_record_3_1c = individual_fit(subtracted_regions3[1][0],subtracted_regions3[1][1],manypeaks,p0_3_1c,bd_3_1c)
fit_image_3_2,fit_record_3_2 = individual_fit(subtracted_regions3[2][0],subtracted_regions3[2][1],gauss,p0_3_2,bd_3_2)

/Users/Amanda/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in divide
  This is separate from the ipykernel package so we can avoid doing imports until


1 spectra did not converge to a solution.
All spectra converged to a fit solution.
9 spectra did not converge to a solution.
2 spectra did not converge to a solution.
All spectra converged to a fit solution.
1 spectra did not converge to a solution.
2 spectra did not converge to a solution.


In [ ]:
sample_plot_waterfall(16, subtracted_regions1[0][0], subtracted_regions1[0][1],fit_image_1_0)
sample_plot_waterfall(16, subtracted_regions1[1][0], subtracted_regions1[1][1],fit_image_1_1)
sample_plot_waterfall(16, subtracted_regions3[0][0], subtracted_regions3[0][1],fit_image_3_0)
sample_plot_waterfall(16, subtracted_regions3[1][0], subtracted_regions3[1][1],fit_image_3_1a)
sample_plot_waterfall(16, subtracted_regions3[1][0], subtracted_regions3[1][1],fit_image_3_1b)
sample_plot_waterfall(16, subtracted_regions3[1][0], subtracted_regions3[1][1],fit_image_3_1c)
sample_plot_waterfall(16, subtracted_regions3[2][0], subtracted_regions3[2][1],fit_image_3_2)

For comparing fitting options of q~3.2 features.

In [ ]:
'''
waterfall_plot(subtracted_regions1[0][1], np.array([subtracted_regions1[0][0],subtracted_regions1[0][0]]).T, loc1+'_nah', title_append=None, log=False)
waterfall_plot(subtracted_regions3[1][1], np.array([subtracted_regions3[1][0],subtracted_regions3[1][0]]).T, loc3+'_nah', title_append=None, log=False)

parameter_trace_display(fit_record_1_0)
parameter_trace_display(fit_record_3_1a)
parameter_trace_display(fit_record_3_1b)
parameter_trace_display(fit_record_3_1c)

sample_plot_waterfall(16, subtracted_regions1[0][0], subtracted_regions1[0][1],fit_image_1_0)
sample_plot_waterfall(16, subtracted_regions3[1][0], subtracted_regions3[1][1],fit_image_3_1a)
sample_plot_waterfall(16, subtracted_regions3[1][0], subtracted_regions3[1][1],fit_image_3_1b)
sample_plot_waterfall(16, subtracted_regions3[1][0], subtracted_regions3[1][1],fit_image_3_1c)
'''

Below:  Experimental code for using decision trees to seek background/feature distinction.  Not presently recommended for use.

In [ ]:
def initial_features_1():
    qq,_ = ref_spec1.T
    features = (((qq > 1.75) & (qq < 1.95)) | ((qq > 3.05) & (qq < 3.4)) | ((qq > 3.55) & (qq < 3.9)))
    return features
def initial_features_2():
    qq,_ = ref_spec1.T
    features = (((qq > 2.6) & (qq < 2.95)) | ((qq > 3.05) & (qq < 3.4)) | ((qq > 4.3) & (qq < 4.7)))
    return features


def grow(arr, nn):
    arr = arr.astype('bool')
    for ii in range(nn):
        arr[1:,:] = arr[1:,:] | arr[:-1,:]
        arr[:-1,:] = arr[1:,:] | arr[:-1,:]
        arr[:,1:] = arr[:,1:] | arr[:,:-1]
        arr[:,:-1] = arr[:,1:] | arr[:,:-1]
    return arr

def local_error_estimate(image):
    error_estimate = np.zeros(image.shape, dtype=float)
    error_estimate[1:-1,1:-1] = (image[1:-1,1:-1] - image[1:-1,2:])**2 \
                                + (image[1:-1,1:-1] - image[1:-1,:-2])**2 \
                                + (image[1:-1,1:-1] - image[2:,1:-1])**2 \
                                + (image[1:-1,1:-1] - image[:-2,1:-1])**2
    error_estimate[0,:] = error_estimate[1,:]
    error_estimate[-1,:] = error_estimate[-2,:]
    error_estimate[:,0] = error_estimate[:,1]
    error_estimate[:,-1] = error_estimate[:,-2]
    error_estimate = (error_estimate / 8.) ** 0.5
    return error_estimate

def twodeebg(intensity, bg_I, legit=None):
    nq, nfiles = intensity.shape
    blank = np.ones((nq, nfiles))
    mult = np.ones(nfiles) #*blank        
    if len(bg_I.shape) == 1:
        bg_I = bg_I.reshape((nq,1))*blank
    if legit is not None:
        mult[legit.any(axis=0)] = ((intensity / bg_I * legit).sum(axis=0)/legit.sum(axis=0,dtype=float)).reshape((nfiles,))[legit.any(axis=0)]  # potential problem line
        if np.any(~legit.any(axis=0)):
            print "Some spectra/timeframes are entirely deselected.  Consider re-tuning your cutoff values."
    else:
        mult = ((intensity / bg_I).mean(axis=0)).reshape((nfiles,))  # potential problem line
    mult = mult.reshape((1,nfiles))
    #diff = intensity - mult*bg_I
    diff = intensity/mult - bg_I
    return mult, diff


def make_bg(image,mult,features):
    nq, nfiles = image.shape
    bg_I = np.zeros(nq)
    mult = mult.flatten()
    #import pdb; pdb.set_trace()
    for ii in range(nq):
        if features[ii,:].any():
            bg_I[ii] = np.median((image[ii,:]/mult)[features[ii,:]])*mult.mean()
        else:
            bg_I[ii] = np.median((image[ii,:]/mult))*mult.mean()
    return bg_I

def tree_bg_fit_adaptive(intensity_image, func, ref, cutoff, tree_depth, max_iter=15, demonstrative=False, loc=None):
    '''"Demonstrative" is like "verbose", but with plots instead of words.
    
    No value for "loc" means no plots saved.
    
    "prediction" nonzero for features.'''
    nq, nfiles = intensity_image.shape
    blank = np.ones((nq, nfiles), dtype=float)
    qq = ref[:, 0]
    index = np.arange(nfiles)
    qq2d = qq.reshape(nq,1)*blank
    X1 = (qq.reshape(nq,1)*blank).flatten()
    X2 = (index.reshape(1,nfiles)*blank).flatten()
    X = np.array(zip(X1,X2))
    # find initial mad guess as to background shape, difference from background,
    # and variance from background-like-ness
    mult = intensity_image.mean(axis=0)
    bg_I = np.median((intensity_image/mult[np.newaxis,:]),axis=1)*mult.mean()
    
    waterfall_plot(intensity_image, ref)
    plt.plot(qq,bg_I,c='k',lw=2)

    mult, diff_image = twodeebg(intensity_image, bg_I)

    waterfall_plot(diff_image, ref,log=False)

    clf = tree.DecisionTreeClassifier(max_depth=tree_depth)
    oldpredictions = []
    
    for ii in range(max_iter):
        if np.isnan(diff_image).any():
            print 'NaNs here, improve coping pls'
        smooth_diff = medfilt(diff_image, 3)
        error_estimate = local_error_estimate(diff_image)
        smooth_err = medfilt(error_estimate, 3)
        #import pdb; pdb.set_trace()
        frame4 = (np.fabs(smooth_diff) / cutoff > smooth_err)
        frame5 = medfilt(frame4, 3)
        frame6 = grow(frame5, 2)
        Y = frame6.flatten()
        clfsol = clf.fit(X, Y)
        prediction = (clfsol.predict(X)).reshape(nq, nfiles)
        '''
        # attempt to deal with unconstrained fits choosing features-only over nonfeatures-only
        if ii == 0:
            test_popt,test_pcov=curve_fit(func,qq2d[~prediction],intensity_image[~prediction])
            test_bg_I = func(qq,*test_popt)
            test_mult, test_diff_image = twodeebg(intensity_image, test_bg_I, ~prediction)
            test_goodness_of_fit = np.abs((test_diff_image[~prediction]).mean())
            inv_prediction = ~prediction
            inv_popt,inv_pcov=curve_fit(func,qq2d[~inv_prediction],intensity_image[~inv_prediction])
            inv_bg_I = func(qq,*inv_popt)
            inv_mult, inv_diff_image = twodeebg(intensity_image, inv_bg_I, ~inv_prediction)
            inv_goodness_of_fit = np.abs((inv_diff_image[~inv_prediction]).mean())
            if inv_goodness_of_fit < test_goodness_of_fit:
                prediction = ~prediction
                print "right side up switched"
            else:
                print "orientation maintained"
        '''
        metrics = []
        for jj in oldpredictions:
            metric = (jj != prediction).sum()
            metrics.append(metric)
        if ii != 0:
            metricbest = min(metrics)
            indexbest = metrics.index(metricbest)
            if metricbest == 0:
                if indexbest == (len(metrics) - 1):
                    print "Converged, iteration %i" % ii
                    suffix = ""
                else:
                    cyclelength = (len(metrics) - indexbest)
                    print "Cyclical convergence, iteration %i, cycle length %i" % (ii, cyclelength)
                    suffix = ", cycle length %i" % cyclelength
                    for kk in range(1, cyclelength):
                        prediction = (prediction | oldpredictions[-kk])
                        oldpredictions.append(prediction)
                break
        oldpredictions.append(prediction)
        
        print "prediction mean", prediction.mean()
        bg_I = make_bg(intensity_image,mult,prediction)
        mult, diff_image = twodeebg(intensity_image, bg_I, ~prediction)

    if demonstrative:
        nplots = len(oldpredictions)
        fig, ax = plotarray(nplots)
        flatax = ax.ravel()
        for ii in range(nplots):
            axii = flatax[ii]
            if ii < nplots:
                axii.tick_params(axis='both', which='both', bottom='off', top='off', labelbottom='off', right='off', left='off',
                                labelleft='off')
                axii.imshow(oldpredictions[ii].T, cmap='binary')
                if ii != (nplots-1):
                    axii.set_title(ii)
                else:
                    axii.set_title("Final")
        fig.suptitle("Evolution of decision tree selection" + suffix)
        fig2, ax2 = plt.subplots(1,2)
        ax2[0].imshow(diff_image.T,cmap='viridis')
        ax2[1].imshow(prediction.T,cmap='binary')
        ax2.axis('off')
        ax2.set_title("Final background-subtracted spectra, feature map")
        if loc is not None:
            fname = (loc + '_dtree_bgselection_evolution.pdf')
            fig.savefig(fname)
            fname2 = (loc + '_diffimage_final.pdf')
            fig2.savefig(fname2)
        #fig.set_figheight(8)
        #fig.set_figwidth(12)
        #return prediction, mult, diff_image, fig, fig2
    return prediction, mult, diff_image

In [ ]:
feature_map1, mult1, final_diff_image1 = tree_bg_fit_adaptive(good_spectra1, quintic, ref_spec1, 3., 5, demonstrative=True, loc=loc1)

In [ ]:
feature_map3, mult3, final_diff_image3 = tree_bg_fit_adaptive(good_spectra3, quintic, ref_spec3, 1.5, 5, demonstrative=True, loc=loc3)

Below:  Write-out code to transfer data to Sam.

In [37]:
def writable_timeseries(spectra,qq=None,indices=None):
    nq, nfiles = spectra.shape
    if qq is None:
        writable = np.zeros((nq+1,nfiles),dtype=float)
    else:
        writable = np.zeros((nq+1,nfiles+1),dtype=float)
        writable[1:,0] = qq
        
    if (qq is not None) and (indices is not None):
        writable[0,0] = np.nan
    if indices is None:
        index = np.arange(nfiles)
    else:
        index = indices        
    if qq is None:
        writable[0,:] = index
    else:
        writable[0,1:] = index
    writable[-nq:,-nfiles:] = spectra
    return writable

outloc = '/Users/Amanda/Dropbox/Sam_data_20170316/supporting/Sam_20170316_csvs/'

fname = outloc+'Full_time_series/Rxn1_full_series.csv'
writable = writable_timeseries(good_spectra1,ref_spec1.T[0,:],kept_indices[0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Full_time_series/Rxn3_full_series.csv'
writable = writable_timeseries(good_spectra3,ref_spec3.T[0,:])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')

fname = outloc+'Local_subtracted_segments/Rxn1_q_3_2.csv'
writable = writable_timeseries(subtracted_regions1[0][1],subtracted_regions1[0][0],kept_indices[0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Local_subtracted_segments/Rxn1_q_3_6.csv'
writable = writable_timeseries(subtracted_regions1[1][1],subtracted_regions1[1][0],kept_indices[0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Local_subtracted_segments/Rxn3_q_2_7.csv'
writable = writable_timeseries(subtracted_regions3[0][1],subtracted_regions3[0][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Local_subtracted_segments/Rxn3_q_3_2.csv'
writable = writable_timeseries(subtracted_regions3[1][1],subtracted_regions3[1][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Local_subtracted_segments/Rxn3_q_4_5.csv'
writable = writable_timeseries(subtracted_regions3[2][1],subtracted_regions3[2][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')

fname = outloc+'Fit_traces/Rxn1_q_3_2_fit.csv'
writable = writable_timeseries(fit_image_1_0,subtracted_regions1[0][0],kept_indices[0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Fit_traces/Rxn1_q_3_6_fit.csv'
writable = writable_timeseries(fit_image_1_1,subtracted_regions1[1][0],kept_indices[0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Fit_traces/Rxn3_q_2_7_fit.csv'
writable = writable_timeseries(fit_image_3_0,subtracted_regions3[0][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Fit_traces/Rxn3_q_3_2_fit_A.csv'
writable = writable_timeseries(fit_image_3_1a,subtracted_regions3[1][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Fit_traces/Rxn3_q_3_2_fit_B.csv'
writable = writable_timeseries(fit_image_3_1b,subtracted_regions3[1][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Fit_traces/Rxn3_q_3_2_fit_C.csv'
writable = writable_timeseries(fit_image_3_1c,subtracted_regions3[1][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')
fname = outloc+'Fit_traces/Rxn3_q_4_5_fit.csv'
writable = writable_timeseries(fit_image_3_2,subtracted_regions3[2][0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n')

head1 = '\t, peak intensity 1, q centroid 1, sigma width 1'
head2 = head1 + ', peak intensity 2, q centroid 2, sigma width 2'
head3 = head2 + ', peak intensity 3, q centroid 3, sigma width 3'

fname = outloc+'Fit_parameters/Rxn1_q_3_2_params.csv'
writable = writable_timeseries(fit_record_1_0,None,kept_indices[0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n', header=head1)
fname = outloc+'Fit_parameters/Rxn1_q_3_6_params.csv'
writable = writable_timeseries(fit_record_1_1,None,kept_indices[0])
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n', header=head1)
fname = outloc+'Fit_parameters/Rxn3_q_2_7_params.csv'
writable = writable_timeseries(fit_record_3_0)
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n', header=head1)
fname = outloc+'Fit_parameters/Rxn3_q_3_2_params_A.csv'
writable = writable_timeseries(fit_record_3_1a)
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n', header=head1)
fname = outloc+'Fit_parameters/Rxn3_q_3_2_params_B.csv'
writable = writable_timeseries(fit_record_3_1b)
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n', header=head2)
fname = outloc+'Fit_parameters/Rxn3_q_3_2_params_C.csv'
writable = writable_timeseries(fit_record_3_1c)
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n', header=head3)
fname = outloc+'Fit_parameters/Rxn3_q_4_5_params.csv'
writable = writable_timeseries(fit_record_3_2)
np.savetxt(fname, writable.T, fmt='%.18f', delimiter=',', newline='\n', header=head1)